In [1]:
import urllib.request
from bs4 import BeautifulSoup
import csv
from time import sleep

In [2]:
#get bin numbers from building footprint csv
with open('footprints/footprint_reduced.csv') as file:
    bin_numbers = [row[4] for row in csv.reader(file)][1:]

In [6]:
all_jobs = []

def queryBIS(bin_number):
    #get html of jobs page using bin_number
    url = "http://a810-bisweb.nyc.gov/bisweb/JobsQueryByLocationServlet?requestid=1&allbin=%s" %(bin_number)
    page = urllib.request.urlopen( url )
    soup = BeautifulSoup(page, "lxml")
    return soup


def getJobsList(bin_number):
    #wait for BIS to allow us to access page
    waiting = True
    while waiting:
        sleep(.1)
        soup = queryBIS(bin_number)
        waiting = soup.find("div", {'id': 'waiting-main'})
    
    #continue if there is an error message 
    error = soup.find("td", {'class' : 'errormsg'})
    if error:
        print('error', bin_number)
        return None
    
    address = soup.find("td", { "class" : "maininfo" }).get_text().replace('Premises: ','')
    content_table = soup.find("table", { "cellspacing" : "1" })
    new_job = []
        
    trs = content_table.findAll("tr")[2:]
        
    for i,row in enumerate(trs):            
        tds = row.findAll("td")
            
        #append to list if next job or last job of content, then reset 
        if len(tds) == 10 or len(trs) == i + 1:
            if new_job:
                all_jobs.append([bin_number, address] + new_job + [' '.join(details)])
            new_job, details = [], []
            
            for td in tds:
                new_job.append(td.get_text().strip()) 
            
        #append to details
        elif len(tds) == 2:
            for td in tds:
                details.append(td.get_text())
                
    return None

In [7]:
for bin_number in bin_numbers:
    getJobsList(bin_number[:100])

error 1058979
error 1057011
error 1083303
error 1079824
error 1058670
error 1059254
error 1061310
error 1060660
error 1060252
error 1081330
error 1055124
error 1060316
error 1061240


HTTPError: HTTP Error 503: Service Unavailable

In [ ]:
header = ['bin','address','file_date','job_num','doc_num','job type',
          'job_status','status_date','lic_num','applicant',
          'in audit','zoning','details']

with open('all_jobs.tsv', 'w') as file:
    writer = csv.writer(file, delimiter ='\t')
    writer.writerow(header)
    for row in all_jobs:
        writer.writerow(row)

In [232]:
import pandas as pd

In [290]:
jobs_df = pd.read_csv('all_jobs.tsv', delimiter='\t')
jobs_df['status_date'] = pd.to_datetime(jobs_df.status_date)

In [291]:
#filter jobs without null values for details
jobs_df = jobs_df[pd.notnull(jobs_df.details)]
#filter ANY SHED JOBS
shed_jobs_df = jobs_df[jobs_df.details.str.contains('SHED')]

In [292]:
list(shed_jobs_df)

['bin',
 'address',
 'file_date',
 'job_num',
 'doc_num',
 'job type',
 'job_status',
 'status_date',
 'lic_num',
 'applicant',
 'in audit',
 'zoning',
 'details']

In [293]:
shed_jobs_df.job_status.value_counts()

R PERMIT-ENTIRE      1754
X SIGNED OFF         1483
P APPROVED             73
H P/E IN PROCESS       57
J P/E DISAPPROVED      11
Q PERMIT-PARTIAL        5
A PRE-FILED             2
3 SUSPENDED             1
B A/P UNPAID            1
Name: job_status, dtype: int64

In [294]:
#filter out jobs that are not approved
shed_jobs_df = shed_jobs_df[~shed_jobs_df.job_status.isin(['3 SUSPENDED','B A/P UNPAID','J P/E DISAPPROVED'])]

In [301]:
#for each unique bin number get the most current SHED job
sheds_job_dates = pd.DataFrame({'bin' : shed_jobs_df.bin.unique()})

def getMostCurrent(row):
    dates_df = shed_jobs_df[shed_jobs_df.bin == row.bin].status_date
    return dates_df.max()

sheds_job_dates['date'] = sheds_job_dates.apply(getMostCurrent, axis = 1)

In [306]:
current_sheds = sheds_job_dates[sheds_job_dates['date'] > '10/14/2016']

In [308]:
current_sheds.to_csv('current_sheds.csv', index=False)